In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS

In [4]:
file = fits.open('HectoMAP_DR16_spec23.fits')
file.info()

Filename: HectoMAP_DR16_spec23.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     87   1148697R x 35C   [22A, D, D, 2A, E, E, 26A, E, E, 2A, E, E, E, E, E, E, 25A, 4A, 8A, 12A, 8A, E, E, E, A, E, 15A, E, 9A, 28A, 34A, 39A, E, E, 4A]   


In [6]:
file[1].header

XTENSION= 'BINTABLE'           / Binary table written by MWRFITS v1.14          
BITPIX  =                    8 / Required value                                 
NAXIS   =                    2 / Required value                                 
NAXIS1  =                  323 / Number of bytes per row                        
NAXIS2  =              1148697 / Number of rows                                 
PCOUNT  =                    0 / Normally 0 (no varying arrays)                 
GCOUNT  =                    1 / Required value                                 
TFIELDS =                   35 / Number of columns in table                     
COMMENT                                                                         
COMMENT  *** End of mandatory fields ***                                        
COMMENT                                                                         
COMMENT                                                                         
COMMENT  *** Column formats 

In [8]:
print(file[1].header['BITPIX'])
print(file[1].header.comments['BITPIX'])

8
Required value


In [10]:
coldefs = file[1].columns

In [12]:
columns = []

In [14]:
for col in coldefs:
    columns.append({
        "Name": col.name,
        "Format": col.format
    })

In [16]:
df = pd.DataFrame(columns)
df

,Name,Format
0,Z_OBJID,22A
1,Z_RA,D
2,Z_DEC,D
3,Z_YJ_ID,2A
4,Z_YJ_Z,E
5,Z_YJ_ZERR,E
6,Z_NED_ID,26A
7,Z_NED_Z,E
8,Z_NED_ZERR,E
9,Z_D11_ID,2A


In [18]:
df_transposed = df.transpose()
df_transposed

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
Name,Z_OBJID,Z_RA,Z_DEC,Z_YJ_ID,Z_YJ_Z,Z_YJ_ZERR,Z_NED_ID,Z_NED_Z,Z_NED_ZERR,Z_D11_ID,...,Z_MMT_CZXCR,Z_MMT_BESTTEMP,Z_MMT_HELIORV,Z_MMT_OBS,Z_MMT_FLD,Z_MMT_SPEC,Z_MMT_FLUX,Z_TOT_Z,Z_TOT_ZERR,Z_TOT_ZSOURCE
Format,22A,D,D,2A,E,E,26A,E,E,2A,...,E,15A,E,9A,28A,34A,39A,E,E,4A


In [20]:
def load_fits_data_as_dataframe(filename):
    with fits.open(filename) as hdul:
        data = hdul[1].data
        header = hdul[1].header

        # Convert the FITS table data to a pandas DataFrame
        df = pd.DataFrame(data.tolist(), columns=data.names)

    return df, header

In [22]:
def plot_fits_with_wcs(filename, xcoord, ycoord, scalar):
    data, header = load_fits_data(filename)
    
    x = data[xcoord]
    y = data[ycoord]
    intensity = data[scalar]
    
    wcs = WCS(header)

    ra, dec = wcs.all_pix2world(x, y, 0)

    plt.figure(figsize=(10, 8))
    plt.scatter(x, y, c=intensity, cmap='viridis', s=1, marker=',')
    plt.colorbar(label='Magnitude Intensity')
    plt.xlabel('Age of the Universe (Gyr)')
    plt.ylabel('R.A. (deg)')
    plt.title('Detected Sources in WCS Coordinates')
    plt.gca().invert_xaxis()
    plt.show()

fits_filename = 'HectoMAP_DR16_spec23.fits'
df_spec, header = load_fits_data_as_dataframe(fits_filename)
df_spec.head()
#plot_fits_with_wcs(fits_filename, 'Z_COS(RA)', 'Z_SIN(RA)’, 'header_3’)